In [1]:
!pip install faiss-cpu pypdf2

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/14.6 MB 1.5 MB/s eta 0:00:10
    --------------------------------------- 0.3/14.6 MB 1.9 MB/s eta 0:00:08
   - -------------------------------------- 0.6/14.6 MB 2.5 MB/s eta 0:00:06
   - -------------------------------------- 0.6/14.6 MB 2.3 MB/s eta 0:00:06
   --- ------------------------------------ 1.1/14.6 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.2/14.6 MB 3.4 MB/s eta 0:00:04
   ---- ----------------------------------- 1.8/14.6 MB 4.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.4/14.6 MB 5.0 MB/s eta 0:00:03
   -------- ------------------------------- 3.1/14.6 MB 5.9 MB/s eta 0:00:02
   --------- ------------------------------ 3.5/14.6 MB 5.8 MB/s eta 0:00:02
   ---------- ----------------------------- 4.0/14.6 MB 5.9 MB/s eta 0:00:02
   ----------

In [2]:
import faiss
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import fitz
import itertools

In [3]:
def load_multiple_pdf_files(file_paths):
    """
    Load multiple PDF files using PyPDFLoader.

    Args:
        file_paths (list): List of file paths to PDF files.

    Returns:
        list: List of raw documents.
    """
    raw_documents = []
    for file_path in file_paths:
        pdf_loader = PyPDFLoader(file_path)
        raw_documents.extend(pdf_loader.load())
    return raw_documents

def split_documents(raw_documents):
    """
    Split raw documents into chunks using RecursiveCharacterTextSplitter.

    Args:
        raw_documents (list): List of raw documents.

    Returns:
        list: List of split documents.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(raw_documents)

# Example usage:
file_paths = [
    "C:/Users/ansutton/Desktop/TPRM/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf",
    "C:/Users/ansutton/Desktop/TPRM/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/APP_SHEET-[SPR-2023] AppSheet SOC 2..pdf"
]

raw_documents = load_multiple_pdf_files(file_paths)
documents = split_documents(raw_documents)

In [2]:
# pdf_file_path = 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf'
# with fitz.open(pdf_file_path) as doc:
#         page_contents = [page.get_text() for page in doc]
pdf_loader = PyPDFLoader("C:/Users/ansutton/Desktop/TPRM/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf")
raw_documents= pdf_loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
documents = text_splitter.split_documents(raw_documents)

In [4]:
embedding_model = OllamaEmbeddings(model='nomic-embed-text',show_progress=True)

In [4]:
batch_size = 50
texts=[doc.page_content for doc in documents]
#document_texts = [doc.page_content for doc in documents]
documents_embeddings = []

In [6]:
for i in range(0, len(texts),batch_size):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embedding_model.embed_documents(batch_texts)
    documents_embeddings.extend(batch_embeddings)


OllamaEmbeddings: 100%|██████████| 50/50 [00:50<00:00,  1.02s/it]

OllamaEmbeddings: 100%|██████████| 50/50 [00:47<00:00,  1.06it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:48<00:00,  1.03it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:41<00:00,  1.19it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:41<00:00,  1.20it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:44<00:00,  1.13it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:42<00:00,  1.18it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:41<00:00,  1.21it/s]

OllamaEmbeddings: 100%|██████████| 50/50 [00:42<00:00,  1.16it/s]

OllamaEmbeddings: 100%|██████████| 30/30 [00:30<00:00,  1.03s/it]


In [5]:
batch_size = 50
texts = [doc.page_content for doc in documents]  # Extracting page content from all documents
documents_embeddings = []

# Process embeddings in batches
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]  # Create a batch of texts
    batch_embeddings = embedding_model.embed_documents(batch_texts)  # Embed the batch
    documents_embeddings.extend(batch_embeddings)  # Extend the list with new embeddings


OllamaEmbeddings: 100%|██████████| 16/16 [00:06<00:00,  2.64it/s]


In [6]:
import numpy as np

In [7]:
#create FAISS index and add the embeddings
document_embeddings_np = np.array(documents_embeddings)
dimension = document_embeddings_np.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(document_embeddings_np)

In [8]:
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document

In [9]:
docstore = InMemoryDocstore(dict(enumerate(documents)))
index_to_docstore_id = {i: i for i in range(len(documents))}

In [10]:
vector_store = FAISS(embedding_model, faiss_index, docstore, index_to_docstore_id)

In [19]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

In [20]:
local_model = "llama3.2" 
llm = ChatOllama(model=local_model)

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever = vector_store.as_retriever(search_kwargs={"k":3}),
    return_source_documents = True
)

In [22]:
def get_answer(query):
    result = qa_chain({"query": query})
    response = result['result']

    source_documents = result['source_documents']
    pages = set()
    citations = []
    for doc in source_documents:
        pages.add(doc.metadata['page'])
        citations.append((doc.metadata['page'], doc.page_content))

    return {
        "response": response,
        "pages": list(pages),
        "citations": citations
    }

In [23]:
query = "What access control procedures are in place?"

In [24]:
structured_answer = get_answer(query)

c:\Users\ansutton\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.93it/s]


In [26]:
print(structured_answer["response"])
print(structured_answer["pages"])
print(structured_answer["citations"])

According to the text, the following access control procedures are in place:

1. Prior to issuing system credentials and granting system access, new internal and external users must be registered and authorized by the entity (CC6.2).
2. For users with administered access, their user system credentials are removed when their access is no longer authorized.
3. Access to production machines, support tools, and network devices is managed via access control lists (ACLs).
4. Modifications to ACLs are recorded and approved by administrators.
5. Inspected access control lists and configurations for group administrator approval requirements are enforced by the access control system prior to provisioning user access to system components.

Additionally, there are references to other Common Criteria controls related to access control:

1. The entity implements logical access security software, infrastructure, and architectures over protected information assets (CC6.1).
2. The customer is responsib

In [ ]:
chain.invoke("When was the access control policy last reviewed?")

In [ ]:
chain.invoke("What is the password management policy in place?")

In [ ]:
chain.invoke("What procedures are followed for decommissioning of equipments?")

In [ ]:
chain.invoke("What steps are taken to ensure removal of client information prior to decommissioning of equipments?")

In [65]:
chain.invoke("When were information security policies and procedures updated?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.83it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]


'The information security policies and procedures were updated on \'1 October 20XX\' (year not clearly mentioned). This information can be found on page 35 of the document, which states: "Information Security Policies and Procedures were updated on 1 October 20XX."\n\nOriginal wording from the document: "Information Security Policies and Procedures were updated on 1 October 20XX."'

In [66]:
chain.invoke("How are emergency changes performed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:11<00:00, 11.35s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 15.54it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.77it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.02s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.55it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 11.36it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.87it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.09it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.32it/s]


'Emergency changes are performed through the Change Management process. According to page 50 of the document, "Emergency Changes: Any change that must be made immediately to avoid a significant impact to service or customer data is considered an emergency change." The process requires documentation and review prior to implementation, as well as post-implementation review and communication to the appropriate stakeholders. It also states that such changes are tracked in the same way as regular changes, with necessary escalation mechanisms in place for urgent issues. \n\nOriginal wording from the document: "Emergency Changes: Any change that must be made immediately to avoid a significant impact to service or customer data is considered an emergency change. Emergency changes follow the same change management process, but are documented and reviewed prior to implementation, implemented, and post-implementation reviewed. The change is communicated to the appropriate stakeholders as per the 

In [67]:
chain.invoke("What cryptographic controls are in place to encrypt the data at rest and in motion?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.17s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.81it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.12it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.23it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.39it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 15.86it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.32it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.76it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 12.28it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]


"In order to find information about cryptographic controls for data encryption at rest and in motion, I would need access to the specific audit report or documentation related to Google Cloud's security measures. Please provide the relevant document or report so that I can locate the required information and respond accordingly. Make sure to mention the page number or position in the document along with the original wording from the source."

In [68]:
chain.invoke("When was the incident management policy last reviewed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 18.18it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.74it/s]


'I am unable to find information about the incident management policy review date in the provided document excerpts. Please provide the complete audit report or the specific section containing details about the incident management policy for further assistance.'

In [69]:
chain.invoke("When was the incident response procedure tested?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.42it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s]


'The incident response procedure was tested during the audit process, specifically during the review of the "Incident Alert and Recording" section on page 51 of the document. The text states, "Log sources are used to generate alerts whenever an anomaly occurs... An anomaly may also be manually documented by a Google employee when an issue is identified or in response to a customer service request." (GCP-[FALL-2023] GCP SOC 2..pdf, p. 51)'

In [ ]:
chain.invoke("How are incidents classified?")

In [ ]:
chain.invoke("What procedures are in place to perform root cause analysis for an incident?")

In [72]:
chain.invoke("What physical perimeter controls are in place?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.08s/it]


"I could not find any information about physical perimeter controls in the provided document excerpts. The mentioned Google Cloud services and features are described at various page numbers, but none specifically address physical perimeter controls. Please provide more context or refer to a different section of the document if you're looking for information on this particular topic.\n\nPage numbers mentioned in the request: 18, 22, 73, and 103."

In [73]:
chain.invoke("What procedures are in place to deploy patches throughout the IT infrastructure?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.27s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 17.27it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.53it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 10.98it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  8.77it/s]


'The document does not specifically mention procedures for deploying patches throughout the IT infrastructure. However, it does discuss security updates and vulnerability management in the context of Google\'s Cloud Security Command Center. This service provides visibility into security and network traffic data across multi-cloud and on-premises environments (page 51). It also offers automated policy enforcement for identifying and responding to threats (page 52). These capabilities help ensure that patches and updates are applied promptly and consistently across the infrastructure, although the specific procedures for patch deployment are not detailed in the provided excerpts.\n\n(Source: "GCP-[FALL-2023] GCP SOC 2..pdf", pages 51-52)'

In [74]:
chain.invoke("How is a back up of the data performed?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.54s/it]


'A backup of the data is performed as per the data retention and deletion policies. The process involves the use of data deletion tools that verify the backup data is deleted following the configured retention period, as part of the deletion mechanism process. This information can be found on page 77 of the document: "was disposed of as per the data retention and deletion policies. No deviations noted. Inspected a sample product and determined data deletion tools verified that backup data was deleted following the configured retention period, as part of the deletion mechanism process. No deviations noted."'

In [75]:
chain.invoke("What backgound checks are performed during hiring of an employee?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.85s/it]


"Background checks performed during hiring of an employee include criminal, credit, and/or security checks on all potential employees, temporary workers, and independent contractors, as well as verification of the individual's education, previous employment, and referrals (Source: Page 43, Document metadata {'source': 'C:/Users/ribhattacharya/Desktop/TPRM-Accelerator/assets/data/Security Evidence Docs/SOC 2/GoogleCloud/Audit-Reports-1720774833381-81ba2e/GCP-[FALL-2023] GCP SOC 2..pdf'}). The specifics or extent of background checks performed depend on the position and location for which the individual is applying."

In [76]:
chain.invoke("What is done to ensure that periodic penetration tests are performed for the infrastructure, devices, and end-points?")


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.60s/it]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 12.04it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 13.63it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


'To ensure that periodic penetration tests are performed for the infrastructure, devices, and end-points, Google has the following measures in place:\n\n1. Penetration tests are performed at least annually. (CC4.1)\n2. The organization performs penetration tests by qualified internal personnel or an external service provider at least annually. (Inquired of the Program Manager)\n\nNo deviations were noted in these procedures during the testing process, as confirmed by the results shared by EY after their inspection. \n\nReference(s):\n- Page 54: "logs is restricted to authorized personnel. Security event logs are monitored continuously using a Google proprietary Security Event Management (SEM) system to detect intrusion attempts and other security related events."\n- Page 168: "Control Description SOC 2 Criteria, Controls, Tests and Results of Tests \\n...112. Penetration tests are performed at least annually. CC4.1..."'